In [1]:
import pandas as pd
import re
import tarfile
import gzip
from bs4 import BeautifulSoup

In [2]:
!pwd

/home/jupyter/notebooks/archive.org/commercial


In [3]:
df = pd.read_json('commercial.json')
df

,archive_id,intervals,video_id
0,CNNW_20160727_000000_Anderson_Cooper_360,"[[0, 12245], [1370837, 1401568], [2355588, 257...",1
1,MSNBCW_20161206_070000_The_11th_Hour_With_Bria...,"[[1016983, 1217033], [1500517, 1716399], [1815...",2
2,CNNW_20161229_130000_New_Day,"[[1019352, 1199349], [1766032, 2004237], [2321...",3
3,CNNW_20161231_220000_CNN_Newsroom_With_Poppy_H...,"[[730563, 890540], [1045111, 1202452], [141094...",4
4,MSNBCW_20150220_180000_Ronan_Farrow_Daily,"[[742842, 932432], [1372005, 1518885], [165642...",5
...,...,...,...
304968,FOXNEWSW_20230715_060000_Hannity,"[[1151150, 1314296], [1870919, 1885900], [2450...",362930
304969,MSNBCW_20230714_100000_Morning_Joe,"[[1081980, 1340906], [2228526, 2480744], [3558...",362931
304970,CNNW_20230715_020000_Whos_Talking_to_Chris_Wal...,"[[0, 66866], [678077, 918317], [1439404, 16847...",362932
304971,FOXNEWSW_20230714_150000_The_Faulkner_Focus,"[[2746360, 3040520], [3345875, 3631010]]",362933


In [4]:
# tar tvzf html-2017.tar.gz > html-2017.txt
archive_id = []
with open('../shared/html-2017.txt') as f:
    for l in f:
        a = l.split('/')
        aid = a[-1].strip().replace('.html.gz', '')
        archive_id.append(aid)

In [5]:
df2017 = pd.DataFrame({'archive_id': archive_id})
df2017

,archive_id
0,
1,CSPAN3_20170601_221600
2,BBCNEWS_20110207_160000_BBC_News
3,KMTP_20160202_020000_News-World_Channel
4,TCN_20010911_070000_Texas_Cable_News
...,...
860231,LINKTV_20160205_223000_France_24
860232,KCSM_20160306_020000_Focus_on_Europe
860233,CSPAN2_20140817_170200_Book_Discussion_on_The_...
860234,FOXNEWSW_20150211_070000_On_the_Record_With_Gr...


In [6]:
cdf = df[df.archive_id.isin(df2017.archive_id)].copy()
cdf

,archive_id,intervals,video_id
0,CNNW_20160727_000000_Anderson_Cooper_360,"[[0, 12245], [1370837, 1401568], [2355588, 257...",1
1,MSNBCW_20161206_070000_The_11th_Hour_With_Bria...,"[[1016983, 1217033], [1500517, 1716399], [1815...",2
2,CNNW_20161229_130000_New_Day,"[[1019352, 1199349], [1766032, 2004237], [2321...",3
3,CNNW_20161231_220000_CNN_Newsroom_With_Poppy_H...,"[[730563, 890540], [1045111, 1202452], [141094...",4
4,MSNBCW_20150220_180000_Ronan_Farrow_Daily,"[[742842, 932432], [1372005, 1518885], [165642...",5
...,...,...,...
200178,MSNBCW_20141125_190000_The_Reid_Report,"[[604837, 822755], [1173071, 1384916], [157931...",244868
200369,MSNBCW_20141224_000000_Hardball_With_Chris_Mat...,"[[0, 18718], [660359, 877443], [1328460, 15119...",245061
200456,MSNBCW_20141229_210000_NOW_With_Alex_Wagner,"[[540106, 729995], [795728, 980579], [1815914,...",245148
212104,MSNBC_20100312_110000_Morning_Joe,"[[0, 34567], [1030530, 1200767], [1646780, 188...",256994


In [7]:
cdf['html_file'] = 'html/' + cdf['archive_id'] + '.html.gz'
cdf

,archive_id,intervals,video_id,html_file
0,CNNW_20160727_000000_Anderson_Cooper_360,"[[0, 12245], [1370837, 1401568], [2355588, 257...",1,html/CNNW_20160727_000000_Anderson_Cooper_360....
1,MSNBCW_20161206_070000_The_11th_Hour_With_Bria...,"[[1016983, 1217033], [1500517, 1716399], [1815...",2,html/MSNBCW_20161206_070000_The_11th_Hour_With...
2,CNNW_20161229_130000_New_Day,"[[1019352, 1199349], [1766032, 2004237], [2321...",3,html/CNNW_20161229_130000_New_Day.html.gz
3,CNNW_20161231_220000_CNN_Newsroom_With_Poppy_H...,"[[730563, 890540], [1045111, 1202452], [141094...",4,html/CNNW_20161231_220000_CNN_Newsroom_With_Po...
4,MSNBCW_20150220_180000_Ronan_Farrow_Daily,"[[742842, 932432], [1372005, 1518885], [165642...",5,html/MSNBCW_20150220_180000_Ronan_Farrow_Daily...
...,...,...,...,...
200178,MSNBCW_20141125_190000_The_Reid_Report,"[[604837, 822755], [1173071, 1384916], [157931...",244868,html/MSNBCW_20141125_190000_The_Reid_Report.ht...
200369,MSNBCW_20141224_000000_Hardball_With_Chris_Mat...,"[[0, 18718], [660359, 877443], [1328460, 15119...",245061,html/MSNBCW_20141224_000000_Hardball_With_Chri...
200456,MSNBCW_20141229_210000_NOW_With_Alex_Wagner,"[[540106, 729995], [795728, 980579], [1815914,...",245148,html/MSNBCW_20141229_210000_NOW_With_Alex_Wagn...
212104,MSNBC_20100312_110000_Morning_Joe,"[[0, 34567], [1030530, 1200767], [1646780, 188...",256994,html/MSNBC_20100312_110000_Morning_Joe.html.gz


In [8]:
html_files = cdf['html_file'].tolist()
html_files[:20]

['html/CNNW_20160727_000000_Anderson_Cooper_360.html.gz',
 'html/MSNBCW_20161206_070000_The_11th_Hour_With_Brian_Williams.html.gz',
 'html/CNNW_20161229_130000_New_Day.html.gz',
 'html/CNNW_20161231_220000_CNN_Newsroom_With_Poppy_Harlow.html.gz',
 'html/MSNBCW_20150220_180000_Ronan_Farrow_Daily.html.gz',
 'html/MSNBCW_20161119_000000_Hardball_With_Chris_Matthews.html.gz',
 'html/FOXNEWSW_20151220_150000_Sunday_Morning_Futures_With_Maria_Bartiromo.html.gz',
 'html/FOXNEWSW_20150104_203000_Americas_News_Headquarters.html.gz',
 'html/MSNBCW_20150324_210000_The_Ed_Show.html.gz',
 'html/MSNBCW_20151209_100000_First_Look.html.gz',
 'html/MSNBCW_20170719_070000_All_In_With_Chris_Hayes.html.gz',
 'html/CNNW_20170224_120000_New_Day.html.gz',
 'html/FOXNEWSW_20160601_080000_The_Kelly_File.html.gz',
 'html/FOXNEWSW_20161116_230000_Special_Report_With_Bret_Baier.html.gz',
 'html/FOXNEWSW_20170425_080000_Special_Report_With_Bret_Baier.html.gz',
 'html/CNNW_20151127_210000_The_Lead_With_Jake_Tapper.

In [9]:
def is_in_intervals(intervals, start, end):
    for (a, b) in intervals:
        if (a <= start <= b) or (a <= end <= b):
            return True
    return False

In [10]:
%%time

i = 0
data = []
ofn = 'archive-cc-2017-nc.csv.gz'
first = True
with tarfile.open('../shared/html-2017.tar.gz', 'r:gz') as tar:
    for tar_info in tar:
        if tar_info.isfile():
            if tar_info.name in html_files:
                fname = tar_info.name
                i += 1
                if i % 100 == 0:
                    # output to file every 100 records
                    print(i, fname)
                    tdf = pd.DataFrame(data)
                    data = []
                    if first:
                        tdf.to_csv(ofn, index=False, header=first, compression='gzip')
                        first = False
                    else:
                        tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')
                try:
                    fobj = tar.extractfile(tar_info)
                    with gzip.GzipFile(fileobj=fobj, mode='rb') as gzip_file:
                        html = gzip_file.read()
                        soup = BeautifulSoup(html, 'html.parser')
                        html = soup.prettify()
                        soup = BeautifulSoup(html, 'html.parser')
                        intervals = cdf[cdf.html_file==tar_info.name]['intervals'].iloc[0]
                        text = ""
                        for a in soup.find_all('div', {'class': 'tvcol'}):
                            l = a.find('a')
                            href = l.attrs['href']
                            m = re.match('/start/(\d+)/end/(\d+)', href)
                            if m:
                                start = int(m.group(1)) * 1000
                                end = int(m.group(2)) * 1000
                                t = a.find('div', {'class': 'snipin nosel'})
                                if not is_in_intervals(intervals, start, end):
                                    text += t.text.strip()
                        archive_id = fname.replace('html/', '').replace('.html.gz', '')        
                        data.append({'archive_id': archive_id, 'text_nc': text})
                except Exception as e:
                    print(e, fname)

# last chunk
tdf = pd.DataFrame(data)
tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')

100 html/CNN_20100104_140000_CNN_Newsroom.html.gz
200 html/CNNW_20160120_140000_CNN_Newsroom_With_Carol_Costello.html.gz
300 html/MSNBCW_20170103_040000_The_11th_Hour_With_Brian_Williams.html.gz
400 html/MSNBCW_20140904_190000_The_Cycle.html.gz
500 html/MSNBCW_20160905_130000_MSNBC_Live.html.gz
'NoneType' object has no attribute 'text' html/MSNBCW_20160228_020000_Citizen_Trump_With_Chris_Matthews.html.gz
600 html/MSNBCW_20150113_080000_All_In_With_Chris_Hayes.html.gz
700 html/CNNW_20161212_170000_CNN_Newsroom_With_Brianna_Keilar.html.gz
800 html/MSNBCW_20161030_070000_In_Other_News.html.gz
'NoneType' object has no attribute 'text' html/MSNBCW_20151223_070000_Citizen_Trump_With_Chris_Matthews.html.gz
900 html/FOXNEWSW_20161030_140000_Sunday_Morning_Futures_With_Maria_Bartiromo.html.gz
1000 html/MSNBCW_20170930_060000_The_11th_Hour_With_Brian_Williams.html.gz
'NoneType' object has no attribute 'text' html/CNNW_20150820_010000_Anthony_Bourdain_Parts_Unknown.html.gz
1100 html/MSNBCW_201412